### **1 - Importing Libraries**

In [ ]:
from unsloth import FastLanguageModel
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
import re
import json
from datetime import datetime

### **2 - Loading Configuration**

In [ ]:
with open('config.json', 'r') as file:
    config = json.load(file)

# general 
HGF = config['general']['HGF']

# output model
output_model_online_Desc = config['outputs']['output_model_online_Desc']
output_model_online_Rec = config['outputs']['output_model_online_Rec']

# model
max_seq_length = config['model']['max_seq_length']
load_in_4bit = config['model']['load_in_4bit']

### **3- Metrics**

In [ ]:
# MRR
def mean_reciprocal_rank(predicted, real):
    ranks = []
    
    for real_value, predicted_list in zip(real, predicted):
        try:
            rank = predicted_list.index(real_value) + 1  # Rank is 1-based
            ranks.append(1 / rank)
        except ValueError:
            ranks.append(0)  # If real_value is not in predicted_list
    
    return (sum(ranks) / len(ranks)) * 100  # Convert to percentage

# NDCG
def dcg_at_k(r, k):
    """ Compute DCG@k given a binary relevance list r (1 if relevant, 0 otherwise). """
    r = np.array(r[:k])  # Consider only top-k predictions
    return np.sum(r / np.log2(np.arange(1, len(r) + 1) + 1))

def ndcg_at_k(predicted, real, k=5):
    """ Compute nDCG@k for a list of predicted rankings and real labels. """
    ndcgs = []

    for real_value, predicted_list in zip(real, predicted):
        # Relevance vector: 1 if correct, 0 otherwise
        relevance = [1 if c == real_value else 0 for c in predicted_list[:k]]

        # Compute DCG and IDCG
        dcg = dcg_at_k(relevance, k)
        idcg = dcg_at_k([1] * min(k, 1), k)  

        # Compute nDCG
        ndcg = dcg / idcg if idcg > 0 else 0
        ndcgs.append(ndcg)

    return np.mean(ndcgs) * 100  # Convert to percentage

### **4 - Loading Data and models**

In [ ]:
test_df = pd.read_csv("Data/test.csv", index_col=0)


model_d, tokenizer_d = FastLanguageModel.from_pretrained(
    model_name = output_model_online_Desc,
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    token = HGF
)


model_r, tokenizer_r = FastLanguageModel.from_pretrained(
    model_name = output_model_online_Rec,
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    token = HGF
)

### **5 - Evaluation**

In [ ]:
promptDesc = """ Below is an instruction that describes a task, paired with an input that provied further context.
Write a response that appropiately completes the request.

### Instruction:
You are an interests analyzer. Based on the following user history, analyze their reading habits and generate a description of what kind of news articles they might be interested in reading next. 

### History:
{}

### Response:
Description : \n
{}

"""

In [2]:
promptRec = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You serve as a personalized news article recommendation system. Based on the user's preference descriptions below and the candidate articles, rank the candidates using their labels.
Output Format:
Ranked News Articles: <START> C#, C#, ..., C# <END>

### Preferences Description:
{}

### Candidates:
{}

### Response:
<think>{} """

In [ ]:
def generate_Descriptions(df,model,tokenizer):

    history = []
    desc = []
    candidates = []
    labels = []

    for (i, row) in df.iterrows():
           
            prompt = row['history']
            inputs = tokenizer([promptDesc.format(prompt, "")], return_tensors="pt").to("cuda")
        
            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens= 900
            )
            response = tokenizer.batch_decode(outputs)
            result = response[0].split("### Response:")[1].split("\nDescription : \n\n\n\n")[1].replace("<｜end▁of▁sentence｜>","")
            
            desc.append(result)
            candidates.append(row['candidate'])
            labels.append(row['label'])
            history.append(prompt)
            
    return history, desc, candidates, labels

In [1]:
def generate_Recommendations(df, model, tokenizer):
    predicted = []
    real = []
    results = []
    candidates = []
    
    for (i, row) in df.iterrows():
            desc = row['Descriptions']
            c = row['Candidates']
            inputs = tokenizer([promptRec.format(desc, c, "")], return_tensors="pt").to("cuda")

            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=1000
            )
            response = tokenizer.batch_decode(outputs)
            result = response[0].split("### Response:")[1]
            match = re.search(r'Ranked News Articles\s*:\s*(.*)', result)
            if match:
                after_phrase = match.group(1)
                cs = re.findall(r'C\d+', after_phrase)

            cs = list(dict.fromkeys(re.findall(r'C\d+', after_phrase) + re.findall(r'C\d+', c)))

            print(i)
                                    
            predicted.append(cs)
            real.append(row['Labels'])
            results.append(result)
            candidates.append(c)

            
    return predicted, real, candidates, results

### **A - Generate Descriptions :**

In [ ]:
h,d,c,l = generate_Descriptions(test_df, model_d, tokenizer_d)

In [ ]:
df = pd.DataFrame({
    "history":h,
    "Descriptions":d,
    "Candidates":c,
    "Labels":l
})

In [ ]:
df['Descriptions'] = df['Descriptions'].str.replace('\n\n', '', regex=False)

In [ ]:
df.to_csv('Data/MIND-Preprocessed/test.csv', index=False)

### **B - Generate Recommendations :**

In [ ]:
predicted, real, candidates, results = generate_Recommendations(df, model_r, tokenizer_r)

In [ ]:
recommendations = pd.DataFrame({
    "Predicted": predicted,
    "Real": real,
    "Candidates" : candidates,
    "Results" : results
})

In [ ]:
recommendations.to_csv("Outputs/Output.csv")

In [ ]:
mrr_score = mean_reciprocal_rank(recommendations["Predicted"], recommendations["Real"])
ndcg_score_5 = ndcg_at_k(recommendations["Predicted"], recommendations["Real"], k=5)
ndcg_score_10 = ndcg_at_k(recommendations["Predicted"], recommendations["Real"], k=10)
_id = datetime.now().strftime("%Y%m%d%H%M%S%f")

In [ ]:
metrics = pd.DataFrame({
    "ID": [_id],
    "MRR": [round(mrr_score,2)],
    "NDCG@5":[round(ndcg_score_5,2)],
    "NDCG@10": [round(ndcg_score_10,2)]
})

In [ ]:
old_metrics = pd.read_csv('Results/metrics.csv', index_col = 0)
metrics = pd.concat([old_metrics, metrics], axis=0, ignore_index=True)
metrics.to_csv('Results/metrics.csv', index=False)